In [1]:
import sqlite3
import requests
import pandas as pd

server = 'https://pubgeo.zwemwater.nl/geoserver/zwr_public/wfs'


In [2]:
body = """
    <GetFeature xmlns="http://www.opengis.net/wfs" service="WFS" version="1.1.0" outputFormat="application/json" xsi:schemaLocation="http://www.opengis.net/wfs http://schemas.opengis.net/wfs/1.1.0/wfs.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
        <Query typeName="zwr_public:zwemplekken" srsName="EPSG:4326" xmlns:zwr_public="https://pubgeo.zwemwater.nl/geoserver/zwr_public">
        </Query>
    </GetFeature>
"""
response = requests.post(server, body)


In [3]:
data = pd\
    .json_normalize(response.json()['features'])\
    .drop_duplicates(subset=["properties.zwemwaterlocatie_id"])\
    .drop(["type", "id", "geometry.type", "properties.key_id", "bbox", "geometry_name", "properties.datum"], axis=1)\
    .rename(columns={
            "properties.zwemwaterlocatie_id": "id",
            "properties.naam": "name",
            "properties.korte_naam": "alternate_name",
            "properties.status": "status",
            "geometry.coordinates": "coordinates"
    })\
    .reindex(columns=["id", "name", "alternate_name", "coordinates", "status",])\
    .set_index("id")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# Split coordinates into columns
data["lat"], data["long"] = [item[1] for item in data['coordinates']], [
    item[0] for item in data['coordinates']]
data.drop("coordinates", axis=1, inplace=True)

In [ ]:
# Clean up names
data["name"] = data["name"]\
    .str.strip()\
    .str.title()\
    .str.replace("^Rcn", "RCN", regex=True)\
    .str.replace("^T ", "'t ", regex=True)\
    .str.replace("['|`]T ", "'t ", regex=True)\
    .str.replace("['|`]S", "'s", regex=True)\
    .str.replace("Ij", "IJ", regex=True)


In [ ]:
data = data.sort_values(by="name")
data

,name,alternate_name,status,lat,long
id,,,,,
7350900,'s-Gravenzande,'s-Gravenzande,goed,52.0126,4.1392
6841460,'t Gasselterveld,'t Gasselterveld,goed,52.9722,6.7498
1029,'t Hilgelo,'t Hilgelo,goed,51.9936,6.7208
1259,'t Hoefsven,'T Hoefsven,goed,51.6761,5.0942
1396,'t Loomeer,`T Loomeer,goed,52.8924,6.7048
...,...,...,...,...,...
961,Zwemplaats Earnewâld,Zwemplaats Earnewâld,goed,53.1315,5.9559
1451,Zwemplas Hommelheide,Zwemplas Hommelheide,goed,51.0678,5.8867
5911123,Zwemplas Woude,Zwemplas Woude,goed,51.8810,4.6143


In [ ]:
locations = data[["name", "alternate_name", "lat", "long"]]
locations

,name,alternate_name,lat,long
id,,,,
7350900,'s-Gravenzande,'s-Gravenzande,52.0126,4.1392
6841460,'t Gasselterveld,'t Gasselterveld,52.9722,6.7498
1029,'t Hilgelo,'t Hilgelo,51.9936,6.7208
1259,'t Hoefsven,'T Hoefsven,51.6761,5.0942
1396,'t Loomeer,`T Loomeer,52.8924,6.7048
...,...,...,...,...
961,Zwemplaats Earnewâld,Zwemplaats Earnewâld,53.1315,5.9559
1451,Zwemplas Hommelheide,Zwemplas Hommelheide,51.0678,5.8867
5911123,Zwemplas Woude,Zwemplas Woude,51.8810,4.6143


In [ ]:
status = data[["status"]].copy()


In [ ]:
status["timestamp"] = pd.Timestamp("now").isoformat()
status["temperature"] = None

In [ ]:
status

,status,timestamp,temperature
id,,,
7350900,goed,2023-08-09T08:22:36.333758,None
6841460,goed,2023-08-09T08:22:36.333758,None
1029,goed,2023-08-09T08:22:36.333758,None
1259,goed,2023-08-09T08:22:36.333758,None
1396,goed,2023-08-09T08:22:36.333758,None
...,...,...,...
961,goed,2023-08-09T08:22:36.333758,None
1451,goed,2023-08-09T08:22:36.333758,None
5911123,goed,2023-08-09T08:22:36.333758,None


In [ ]:
db = sqlite3.connect("../dataset.sqlite3")
locations.to_sql("locations", db, if_exists="append")
status.to_sql("status", db, if_exists="append")


743